In [1]:
import h2o

In [2]:
h2o.init(max_mem_size="25g", nthreads=-1)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_181"; OpenJDK Runtime Environment (build 1.8.0_181-8u181-b13-0ubuntu0.16.04.1-b13); OpenJDK 64-Bit Server VM (build 25.181-b13, mixed mode)
  Starting server from /home/paperspace/anaconda3/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp9qf57kt4
  JVM stdout: /tmp/tmp9qf57kt4/h2o_paperspace_started_from_python.out
  JVM stderr: /tmp/tmp9qf57kt4/h2o_paperspace_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.7
H2O cluster version age:,"28 days, 3 hours and 9 minutes"
H2O cluster name:,H2O_from_python_paperspace_xz50ue
H2O cluster total nodes:,1
H2O cluster free memory:,22.22 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"accepting new members, healthy"


In [3]:
import sqlite3

In [4]:
!ls ../Databases

reviewsV1db  reviewsV1.db  reviewsV2.db  reviewsV3.db


In [5]:
conn = sqlite3.connect('../Databases/reviewsV1.db')

In [6]:
import pandas as pd

In [7]:
with sqlite3.connect('../Databases/reviewsV1.db') as conn:
    data = pd.read_sql_query('SELECT * FROM Review', conn)

In [8]:
data.sort_values(by='Time', inplace=True)
data.reset_index(drop=True, inplace=True)
TRAIN_SIZE = int(data.shape[0] * 0.7)
TEST_SIZE = data.shape[0] - TRAIN_SIZE

In [9]:
TRAIN_SIZE

254883

In [10]:
TEST_SIZE

109236

In [50]:
data_train = data[0: TRAIN_SIZE]
data_test = data[TRAIN_SIZE:]

In [12]:
from h2o.estimators.word2vec import H2OWord2vecEstimator

In [13]:
from h2o import H2OFrame

In [14]:
data_train = H2OFrame(data_train)

/home/paperspace/anaconda3/lib/python3.6/site-packages/h2o/utils/shared_utils.py:177: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = _handle_python_lists(python_obj.as_matrix().tolist(), -1)[1]


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [15]:
corpus = data_train['Text'].tokenize("\\W+")

In [16]:
w2v_model = H2OWord2vecEstimator(vec_size = 100, model_id = "w2v.hex")
w2v_model.train(training_frame=corpus)

word2vec Model Build progress: |██████████████████████████████████████████| 100%


In [17]:
!ls

Avg W2V.ipynb	      BOW -- Logistic Regression.ipynb	TFIDF.ipynb
best.model	      README.md				TFIDF W2v.ipynb
best_w2v_tfidf.model  tfidf_best.model			w2v.hex


In [26]:
w2v_model.find_synonyms("tasty", count = 5)

OrderedDict([('pack', 0.7622376084327698),
             ('although', 0.7599578499794006),
             ('super', 0.7568118572235107),
             ('prefer', 0.7419950366020203),
             ('airplane', 0.7144638299942017)])

In [32]:
w2v_model.find_synonyms("spent", count = 5)

OrderedDict([('looked', 0.8137639760971069),
             ('hours', 0.8076111078262329),
             ('somehow', 0.7911955714225769),
             ('things', 0.7813720107078552),
             ('horrified', 0.7732985615730286)])

In [33]:
Dtrain = w2v_model.transform(corpus, aggregate_method = "AVERAGE")  # Performs average W2V

In [34]:
Dtrain.shape

(254883, 100)

In [40]:
Dtrain['Label'] = data_train["Polarity"].asfactor()

In [51]:
data_test = H2OFrame(data_test)
corpus = data_test['Text'].tokenize("\\W+")

Dtest = w2v_model.transform(corpus, aggregate_method = "AVERAGE")


Dtest['Label'] = data_test["Polarity"].asfactor()
# Dtest.shape

/home/paperspace/anaconda3/lib/python3.6/site-packages/h2o/utils/shared_utils.py:177: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = _handle_python_lists(python_obj.as_matrix().tolist(), -1)[1]


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [35]:
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [37]:
hyper_params = {
    'alpha': [0, .25, .5, .75, 1],
    'lambda': [1000, 100, 10, 1, 0, 0.1, 0.01, 0.001, 0.0001, 0.00001]
}

In [38]:
estimator = H2OGeneralizedLinearEstimator(family = 'binomial', # Binomial for logistic regression
                                    nfolds=10,
                                    keep_cross_validation_predictions=True,  # For getting the cv results
                                    fold_assignment="stratified",    # For unbalanced class labels
                                    standardize = True,   # Standardize the data before modelling
                                    seed=42)

In [39]:
grid = H2OGridSearch(model = estimator, 
                     hyper_params = hyper_params,
                     search_criteria = {'strategy': "Cartesian"}) # Cartesian for fitting all the hp combinations

In [41]:
grid.train(y = "Label", training_frame = Dtrain)

glm Grid Build progress: |████████████████████████████████████████████████| 100%


In [42]:
param_combs = []
for ids in grid.model_ids:
    param_combs.append(grid.get_hyperparams_dict(ids, False))

cv_results = []
for iters, param in enumerate(param_combs):
    res = grid._grid_json['cross_validation_metrics_summary'][iters].as_data_frame()
    res.set_index('', inplace=True)
#     print(param)
    cvRes = res.T[['accuracy', 'f1', 'recall', 'precision']].loc['mean']
    cvRes['alpha'] = param['alpha']
    cvRes['lambda'] = param['lambda']
    cv_results.append(cvRes)

In [43]:
results = pd.DataFrame(cv_results).reset_index(drop=True)

In [45]:
results.sort_values(by='f1', ascending=False)

,accuracy,f1,recall,precision,alpha,lambda
9,0.8970393,0.9411353,0.9673946,0.91630155,0.00,0.00001
4,0.8971495,0.9411336,0.9663495,0.9172169,1.00,0.00001
5,0.89686656,0.94113,0.9689185,0.9149249,0.50,0.00001
3,0.89704335,0.9411288,0.9672499,0.91641736,0.75,0.00001
12,0.8969846,0.94112825,0.96778834,0.915936,0.75,0.00000
11,0.8969846,0.94112825,0.96778834,0.915936,1.00,0.00000
14,0.8969846,0.94112825,0.96778834,0.915936,0.00,0.00000
13,0.8969846,0.94112825,0.96778834,0.915936,0.25,0.00000
10,0.8969846,0.94112825,0.96778834,0.915936,0.50,0.00000
8,0.89705485,0.94112045,0.96699715,0.9166213,0.25,0.00001


So as we can see that the best value of F1 score is from the alpha = 0 and the lambda = 0.00001

In [46]:
clf = H2OGeneralizedLinearEstimator(family = 'binomial', # Binomial for logistic regression
                                    nfolds=0,
                                    keep_cross_validation_predictions=False,
                                    standardize = True,
                                    seed=42,
                                    alpha=0,
                                    lambda_=0.00001)

In [47]:
clf.train(y='Label', training_frame=Dtrain)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [52]:
Dtest.head(5)

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100,Label
-0.0765935,0.0379043,0.0764101,-0.0334471,-0.103389,-0.0829096,0.10944,0.0295401,-0.0679619,0.0503358,0.106777,-0.0112015,0.163895,-0.10535,-0.0632288,0.0185676,-0.0865714,0.132278,0.160321,0.0568595,-0.0284776,-0.0534351,-0.17403,-0.145002,0.0798052,-0.0646833,-0.0222424,-0.162608,-0.106835,0.116643,-0.0326197,-0.186726,0.0447839,-0.00908741,-0.265221,-0.0500213,-0.101119,0.0141155,-0.0856293,-0.139665,-0.0266627,-0.15129,-0.0653463,-0.0376328,0.0497207,0.106536,-0.109644,0.0406661,0.104839,0.0172098,-0.0259079,-0.0532858,0.000273317,0.124068,-0.114546,-0.025198,0.006918,-0.0707046,0.105556,0.0278773,-0.0785708,0.142715,0.0322835,-0.11158,0.0826041,0.0488086,-0.0228256,0.0144346,-0.0889524,-0.0315468,0.0140244,-0.184668,-0.00724963,0.156506,-0.159669,-0.0428037,0.15405,0.0490119,0.0487271,0.0969412,0.00928603,0.109724,0.0553607,0.154458,-0.0410932,-0.121036,-0.0774676,0.0513469,-0.0224081,-0.222832,0.0353577,0.0277342,-0.128045,0.226016,0.109727,-0.0421792,-0.0920685,-0.0589863,0.027289,0.189465,positive
-0.116857,-0.175938,-0.0217791,-0.107424,-0.0246777,0.080834,0.016704,-0.038985,0.100121,-0.0664955,0.182505,0.0521783,0.193063,-0.00682304,0.0130215,-0.0115674,0.0203457,0.179497,0.118615,0.086758,-0.0382497,0.0494959,-0.0975104,-0.0831707,0.188206,-0.0415862,-0.0386754,0.0832397,-0.212208,0.0889459,-0.0961323,-0.106799,-0.0432385,0.0743574,0.189078,0.0331322,-0.141422,0.0689001,-0.0173589,-0.170661,-0.0263197,-0.192177,0.0925974,0.144326,0.0299602,0.305129,-0.0917149,0.220735,0.0296063,0.0834692,-0.0272097,0.0599483,-0.11051,0.0111282,-0.102759,0.0863649,-0.0382939,-0.0319427,-0.0063921,0.09897,0.0719474,0.0755308,-0.159237,0.248362,0.183083,-0.0287423,-0.128303,0.113001,-0.0447961,0.102846,0.122104,-0.0592526,0.0771601,0.0587099,-0.0818285,0.0187138,0.0467672,-0.179872,-0.0599967,-0.0811749,0.162581,0.023635,0.0807669,-0.0493626,-0.0614078,-0.152375,0.195187,0.0244012,0.0888725,-0.0880928,-0.00871245,-0.0292957,-0.331218,0.101054,0.0696838,-0.0134588,-0.0419256,0.0256919,0.00381915,0.0285296,positive
-0.0447095,-0.089861,0.0846873,-0.0671369,-0.0530149,0.0123574,0.0505329,-0.00429256,-0.0647947,0.0247184,0.162776,-0.00797652,0.0678325,-0.0676666,-0.0122785,0.0351833,-0.0309391,0.15475,0.11792,-0.0322082,-0.0571661,0.0371076,-0.120914,-0.0564893,0.164983,-0.0880566,0.0453494,0.0220761,-0.0994264,0.103477,-0.0284554,-0.105584,-0.128956,0.00412661,-0.0364055,-0.0025346,-0.0683706,0.119997,-0.0127588,-0.0457434,-0.0259813,-0.136582,-0.0853951,0.0806476,0.0645884,0.117814,-0.0803176,0.0742157,0.0209467,0.13954,-0.00868126,0.00378794,-0.00646825,0.0632147,-0.0229065,0.0399529,0.061734,-0.0182689,0.0545446,-0.00761769,-0.0620736,0.149304,-0.00892214,-0.00582368,0.00915704,0.00648101,-0.0570286,0.127869,-0.143423,-0.0200862,0.11646,-0.108078,0.112189,0.142629,-0.0658562,-0.0232083,0.0607203,-0.0964652,-0.0288932,0.0111387,0.113943,0.0770542,-0.00282992,-0.023201,-0.169986,-0.0801803,0.0155951,-0.0150505,0.0338897,-0.261919,0.0838546,-0.0245813,-0.149399,0.211501,0.144119,-0.0947624,-0.00918434,-0.0270127,0.0450569,0.104252,positive
-0.0623591,-0.181892,0.045564,-0.00275936,-0.107872,0.0407679,0.109687,0.0606924,-0.0618132,0.0213366,0.12917,0.0185235,0.102043,-0.00112805,-0.0353344,-0.0193064,-0.00891635,0.00677017,0.173543,0.0203019,-0.0566748,0.0159742,-0.133832,-0.0523132,0.049097,-0.101234,0.0810843,0.0163788,-0.191182,-0.0172574,-0.101619,-0.143912,0.0237711,0.00394092,-0.139448,-0.0353488,-0.0952156,0.125041,-0.0722101,-0.024517,0.00493623,-0.196915,-0.178299,-0.0626713,0.0489228,0.0532148,0.0149418,0.0494924,0.115791,0.017074,0.035857

In [1]:
from prettytable import PrettyTable

mp = clf.model_performance(Dtest, train=False, valid=False)

f1 = mp.F1()[0][1]

ac = mp.accuracy()[0][1]

pre = mp.precision()[0][1]
re = mp.recall(thresholds=[mp.find_threshold_by_max_metric(metric='recall')])[0][1]

x = PrettyTable()

x.field_names = ['Vectorization', 'Accuracy', 'Precision', 'Recall', 'F1']
x.add_row(['Avg. W2v', ac, pre, re, f1])
x.add_row(['BOW', 0.8963253872349775, 0.9077152225405725, 0.9733473824312333, 0.9393863165613174])
x.add_row(['TFIDF', 0.899932256765169, 0.9132666367610086, 0.9709738243123336, 0.9412365536483224])
x.add_row(['w2v-TFIDF', 0.8726518730088981, 0.883704076497232, 0.973855556664781, 0.9265921911526467])
print(x)

+---------------+--------------------+--------------------+--------------------+--------------------+
| Vectorization |      Accuracy      |     Precision      |       Recall       |         F1         |
+---------------+--------------------+--------------------+--------------------+--------------------+
|    Avg. W2v   |  0.88492804569922  | 0.9978506179473401 |        1.0         | 0.9320202582549146 |
|      BOW      | 0.8963253872349775 | 0.9077152225405725 | 0.9733473824312333 | 0.9393863165613174 |
|     TFIDF     | 0.899932256765169  | 0.9132666367610086 | 0.9709738243123336 | 0.9412365536483224 |
|   w2v-TFIDF   | 0.8726518730088981 | 0.883704076497232  | 0.973855556664781  | 0.9265921911526467 |
+---------------+--------------------+--------------------+--------------------+--------------------+


In [85]:
mp.confusion_matrix()

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5809080450790592: 


,negative,positive,Error,Rate
negative,10442.0,8634.0,0.4526,(8634.0/19076.0)
positive,3943.0,86217.0,0.0437,(3943.0/90160.0)
Total,14385.0,94851.0,0.1151,(12577.0/109236.0)


In [86]:
h2o.cluster().shutdown()

H2O session _sid_99ef closed.
